Note: It is important to specify the versions of libraries used after getting the code to run first time. These libraries are updated quite often and this usually results in error and incompatibilty between liberaries.

Use: "pip freeze > requirements.txt" to generate a file containing all the details about libraries used and their versions. This will come handy in case of replicating code.

On kaggle keep the environment pinned to original to avoid conflict in future


In [1]:
!pip install -q torch==2.0.0 bitsandbytes transformers==4.36.0 accelerate==0.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 93.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 84.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.

In [4]:
# Standard code to load LLM using Huggingface libraries
# llama-2 7b-chat is used for both tokenization and generation
# To access llama-2 in kaggle go to +Add input -> Models -> llama-2
# Copy it's path here as base_model_name

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

base_model_name = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"

# This is used to break the model down to reduce memory usage passed on to the LLM loading function
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Here the LLM is configuration are set
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)

2024-03-20 19:32:43.865031: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 19:32:43.865137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 19:32:43.996406: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [6]:
# dataset imported to test base LLM
import pandas as pd
df=pd.read_csv("/kaggle/input/diabetes-qa/QA_dataset - Sheet1.csv")
df = df.tail(40)

# Pipeline defined to generate responses from LLM
# queries are passed on to this function to generate reponses
# max_length should be adjusted according to data
text_gen = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=350)


In [13]:
# Query in  string format passed on to the pipeline function to generate response
output = text_gen(df["Questions"].iloc[7])
print(output[0]['generated_text'])

Can adults develop type 1 diabetes, and if so, how is it diagnosed?
 nobody knows for sure why some people develop type 1 diabetes, but it's believed to involve a combination of genetic and environmental factors. Type 1 diabetes can occur at any age, including in adulthood. In fact, type 1 diabetes is sometimes referred to as "latent autoimmune diabetes in adults" (LADA) because it can develop in adults who were previously healthy and did not experience any symptoms until later in life.

Adults can develop type 1 diabetes, and the symptoms are similar to those in children and teens. They may include:

* Increased thirst and urination
* Fatigue
* Blurred vision
* Slow healing of cuts and wounds
* Tingling or numbness in the hands and feet
* Recurring skin, gum, or bladder infections
* Flu-like symptoms, such as weakness, dizziness, and stomach pain


